STAT 2255: Homework 04
======================



## Problem 1 (50 points)



### part (a)



To check if there is any duplicates in a list, we have covered 3
different solutions. Can you come up with a solution that is more
efficient than them?

**Hint**: sorting a list with `L.sort` takes $n\log_{2}n$ time.



### Sample solution



In [1]:
def check2(L, item, left = 0, right = None):
    if right is None: 
        right = len(L)
    if right - left == 0: # empty list
        return False
    if right - left == 1: # list of a single element
        return L[left] == item
    median = (right + left) // 2
    if item < L[median]:
        return check2(L, item, left, median)
    else:
        return check2(L, item, median, right)
    
def duplicates4(L):
    L.sort()
    n = len(L)
    for i in range(1,n):
        if check2(L, L[i], 0, i):
            return True
    return False

### part (b)



For `n = [800, 1600, 3200, 6400, 12800]`, use the timetrials function we
defined in the notebook, calculate the running time for **duplicates2**
and your function to verify that your function is indeed more efficient.



### Sample solution



In [2]:
def timetrials(func, n, trials = 10):
    totaltime = 0
    L = list(range(n))
    for i in range(trials):
        start = time.time() 
        func(L)
        time_for_one_run = time.time() - start
        totaltime += time_for_one_run
    print("average = %10.7f for n = %d" % (totaltime/trials, n))

def duplicates2(L):
    n = len(L)
    for i in range(1,n):
        for j in range(i):
            if L[i] == L[j]:
                return True
    return False

import time

print("Times for duplicates2:")
for n in [800, 1600, 3200, 6400, 12800]:
    timetrials(duplicates2, n)

print("\nTimes for duplicates4:")
for n in [800, 1600, 3200, 6400, 12800]:
    timetrials(duplicates4, n)

Times for duplicates2:
average =  0.0076936 for n = 800
average =  0.0200237 for n = 1600
average =  0.0834360 for n = 3200
average =  0.3538107 for n = 6400
average =  1.4217803 for n = 12800

Times for duplicates4:
average =  0.0004680 for n = 800
average =  0.0011002 for n = 1600
average =  0.0024213 for n = 3200
average =  0.0052920 for n = 6400
average =  0.0115125 for n = 12800


## Problem 2 (50 points)

The Cauchy($\theta$, 1) distribution has the following probability
density function

$$f(x\,|\,\theta) = \frac{1}{\pi[1+(x-\theta)^2]},$$

where $\theta$ is the unknown parameter.

Suppose $x_1$, $x_2$, $\dots$, $x_n$ are observed independent
sample from the Cauchy distribution, then the loglikelihood function for
the data can be written as:

$$\ell(\theta) = -n\log \pi - \sum_{i=1}^n \log (1+(x_i-\theta)^2).$$

The maximum likelihood estimator (mle) for $\theta$, denoted as
$\theta_0$, is then the solution of the following equation

$$\ell'(\theta) = \frac{d\ell(\theta)}{d\theta} = -2\sum_{i=1}^n \frac{\theta-x_i}{1+(\theta-x_i)^2} = 0.$$

Then, the root finding techniques we have learned can be used here.
Essentially, we are looking for $\theta_0$ such that
$\ell'(\theta_0) = 0$.



### Part (a)

Now, suppose our observed data $x_i$'s are defined in a list called
data such that:



In [3]:
data = [-2.44, 3.29, 3.71, -2.40, 4.53, -0.07, -1.05, -13.87, -2.53,
        -1.75, 3.30, 1.77, -0.23, 2.76, 3.80, 3.47, 56.75, -1.34, 4.24]

Use the **newton method** you wrote during the class, find
the maximizer for the log-likelihood function.

**Hint**:

1.  The newton method requires the derivate of function $\ell'$, $\ell''$.
    The expression for $\ell''$ is given below:
    $$\ell''(\theta) = -2\sum_{i=1}^n \frac{1-(\theta-x_i)^2}{[1+(\theta-x_i)^2]^2}$$
2.  Given the $x_i$'s in list called data, if you want to write a
    function such that

    $$g(\theta) = \sum_{i=1}^n (\theta-x_i)^2,$$

    one possible way is

```python
    def g(theta):
        return(sum([(theta-x)**2 for x in data]))
```
3.  SET **init = 4** and set **max<sub>iter</sub> = 100**.


In [1]:
data = [-2.44, 3.29, 3.71, -2.40, 4.53, -0.07, -1.05, -13.87, -2.53,
        -1.75, 3.30, 1.77, -0.23, 2.76, 3.80, 3.47, 56.75, -1.34, 4.24]

def lp(theta):
    return(-2 * sum([(theta - x) / (1 + (theta - x)**2) for x in data]))

def lpp(theta):
    return(-2 * sum([(1 - (theta - x)**2) / (1 + (theta - x)**2)**2 for x in data]))

def newton(f, f_prime, init, epsilon = 10**(-5), max_iter=100):
    x_old = init
    counter = 0
    while counter < max_iter:
        try:
            step_size = f(x_old) / f_prime(x_old)
        except ZeroDivisionError:
            print("f_prime eveluate to 0 during iteration")
            return None
        x_new = x_old - step_size
        if (abs(step_size) < epsilon):
            return x_new
        x_old = x_new
        counter += 1
    print("Fail to get converged result")

newton(lp, lpp, 4)

3.109774666820391

### Part (b)

Use the **Fixed point** method to solve the problem in part (a).

**Hint**:

1.  try two different values for **alpha = -0.25 and 0.125**.
2.  set **init = 4**.
3.  set **max<sub>iter</sub> = 100**.

In [5]:
def fixpoint(f, init, alpha, epsilon = 10**(-5), max_iter=100):
    x_old = init
    counter = 0
    while counter < max_iter:
        x_new = x_old + alpha * f(x_old)
        if (abs(x_new-x_old) < epsilon):
            return x_new
        x_old = x_new
        counter += 1
    print("Fail to get converged result")
    
print(fixpoint(lp, 4, -0.25))
print(fixpoint(lp, 4, 0.125))

Fail to get converged result
None
3.109775572505576
